In [2]:
temp = !pwd
current_dir = temp[0]
print("current dir:", current_dir)

current dir: /home/rink/work/github/ckb-notebook


In [3]:
# contract always success
contract_src_path = current_dir + "/contracts/always_success.c"
print("contract src path: " + contract_src_path)
!echo -e "int main() {\n\
    return 0;\n\
}" > contracts/always_success.c

contract src path: /home/rink/work/github/ckb-notebook/contracts/always_success.c


In [4]:
# build contract
contract_bin_path = current_dir + "/build/always_success"
print("contract bin path:", contract_bin_path)
!riscv64-unknown-elf-gcc -O3 -mcmodel=medlow -Wall -Werror -Wno-nonnull-compare -Wno-unused-function -Wl,-static -fdata-sections -ffunction-sections -Wl,--gc-sections -Wl,-s -o {contract_bin_path} {contract_src_path}

contract bin path: /home/rink/work/github/ckb-notebook/build/always_success


In [5]:
# connect rpc wrapper
# usage https://github.com/rink1969/ckb-rpc-wrapper
from tinyrpc.protocols.jsonrpc import JSONRPCProtocol
from tinyrpc.transports.http import HttpPostClientTransport
from tinyrpc import RPCClient

rpc_client = RPCClient(
    JSONRPCProtocol(),
    HttpPostClientTransport('http://127.0.0.1:8999/'))

remote_server = rpc_client.get_proxy()

system_script = remote_server.systemScript()
print("system script info:", system_script)

system script info: {'name': 'system', 'elf_path': 'system', 'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8', 'hash_type': 'type', 'tx_hash': '0xace5ea83c478bb866edf122ff862085789158f5cbff155b7bb5f13058555b708', 'index': '0x0', 'dep_type': 'dep_group'}


In [6]:
# index 0 account for delpoy contract
account_0 = remote_server.getHDUserInfo(0)
print("account 0 info:", account_0)

account 0 info: {'privkey': '0xbb918557dad030f8c16d2cebb3da66d380e11ede53214d6e8097d4967bed9d3c', 'pubkey': '0x025575c6dc037c40b2f7e5cccdb2f071135fe60dfbe1c85bf5a6052734bab7eaaa', 'blake160': '0x32d0281300319169a9c6f01f1bd6ee566089bdfe', 'address': 'ckt1qyqr95pgzvqrrytf48r0q8cm6mh9vcyfhhlq4hj46t'}


In [7]:
# send some capacity to account 0 before 
always_sucess_info = remote_server.deployContract(account_0["privkey"] ,contract_bin_path)
print("always sucess contract info:", always_sucess_info)

always sucess contract info: {'name': 'always_success', 'elf_path': '/home/rink/work/github/ckb-notebook/build/always_success', 'code_hash': '0x0274eb897aef04482f737d3fbee9c5983e510622c6cecd78f545e433ae6e70f0', 'hash_type': 'data', 'tx_hash': '0xf232c8eca2e0db16898650ae924bd5a93ac274c8267e37295197174a5c09c858', 'index': '0x0', 'dep_type': 'code'}


In [9]:
# index 1 account for test
# send some capacity to account index 1
account_1 = remote_server.getHDUserInfo(1)
print("account 1 info:", account_1)

account 1 info: {'privkey': '0x2f3ef8c71d1eea37ccb923d3391ceff6fbf9bc53a797bfd66cf4430e031ba27f', 'pubkey': '0x025d75e3426b74fc6c832dcb3469ae9374be6173f1940cfb14597b5a6c52c16931', 'blake160': '0xfd03fd824f9b777f3671ab776bd694cc76e3c09d', 'address': 'ckt1qyq06qlasf8ekamlxec6kamt662vcahrczwsfucj4q'}


In [18]:
def example_tx():
    tx = {"version":"0x0",
          "cell_deps":[
              {"out_point":
                   {"tx_hash":"0xe02c8a20a64a336cac1e477b9847888251a30a9bfe2d9050ca7ab59f507b15c7",
                    "index":"0x0"},
               "dep_type":
                   "dep_group"}
          ],
          "header_deps":[],
          "inputs":[
              {"previous_output":
                   {"tx_hash":"0xb364c95ba1170b640d72c514f05389c6b37580568e82afb81b28e06935b6309e",
                    "index":"0x0"},
               "since":"0x0"}
          ],
          "outputs":[
              {"capacity":"0x2cb417800",
               "lock":
                   {"code_hash":"0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8",
                    "args":"0xe73ac97da522fbe645811dc5dbfab830d3ca550f",
                    "hash_type":"type"}
              }
          ],
          "outputs_data":["0x"],
          "witnesses":["0xc50477e52d648990415dcb6a0adb9c21df1413e3d738d9001d22b5882398406a01e35a615a26e9713fddfd8840590f756880fdabc4a88ca0c2086a6f074fd9ee00"]
         }
    cell_dep = {"out_point": {"tx_hash": system_script["tx_hash"], "index": system_script["index"]}, "dep_type": system_script["dep_type"]}
    tx["cell_deps"] = [cell_dep]
    return tx
    
tx = example_tx()
print(tx)

import json
tx_json = json.dumps(tx)
print(tx_json)

{'version': '0x0', 'cell_deps': [{'out_point': {'tx_hash': '0xace5ea83c478bb866edf122ff862085789158f5cbff155b7bb5f13058555b708', 'index': '0x0'}, 'dep_type': 'dep_group'}], 'header_deps': [], 'inputs': [{'previous_output': {'tx_hash': '0xb364c95ba1170b640d72c514f05389c6b37580568e82afb81b28e06935b6309e', 'index': '0x0'}, 'since': '0x0'}], 'outputs': [{'capacity': '0x2cb417800', 'lock': {'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8', 'args': '0xe73ac97da522fbe645811dc5dbfab830d3ca550f', 'hash_type': 'type'}}], 'outputs_data': ['0x'], 'witnesses': ['0xc50477e52d648990415dcb6a0adb9c21df1413e3d738d9001d22b5882398406a01e35a615a26e9713fddfd8840590f756880fdabc4a88ca0c2086a6f074fd9ee00']}
{"version": "0x0", "cell_deps": [{"out_point": {"tx_hash": "0xace5ea83c478bb866edf122ff862085789158f5cbff155b7bb5f13058555b708", "index": "0x0"}, "dep_type": "dep_group"}], "header_deps": [], "inputs": [{"previous_output": {"tx_hash": "0xb364c95ba1170b640d72c514f05389c6b3758

In [19]:
# transfer some capacity to a new cell which locked by always_success
# 1. compute lock hash of account 1
account_1_lock_hash = remote_server.lockHash(system_script["code_hash"], account_1["blake160"], system_script["hash_type"])
print("lock hash of account 1:", account_1_lock_hash)

lock hash of account 1: 0x0cdd69779c1bb354913a84b5ed4a41fffdabc00821b76860b84884403d3596e3


In [20]:
# 2. collect live cells to supply enough capacity
transfer_capacity = 1000 * 10 ** 8
print("transfer capacity:", transfer_capacity, "shannon")
fee = 2048 * 1000
print("fee:", fee, "shannon")
inputs_cells = remote_server.queryLiveCellsByCapacity(account_1_lock_hash, transfer_capacity + fee)
print(inputs_cells)
# set tx inputs
tx["inputs"] = inputs_cells["inputs"]

transfer capacity: 100000000000 shannon
fee: 2048000 shannon
{'inputs': [{'previous_output': {'tx_hash': '0x4f821ac07898673d2de8da1d9d6c797f43c82fa4d0537a05cec71e8d0a3d9fd8', 'index': '0x1'}, 'since': '0x0'}], 'capacity': '799997952000'}


In [21]:
# build outputs: transfer and change
transfer_output = {"capacity":str(hex(transfer_capacity)),
               "lock":
                   {"code_hash": always_sucess_info["code_hash"],
                    "args": account_1["blake160"],
                    "hash_type":always_sucess_info["hash_type"]}
        }
print("transfer output:", transfer_output)
change_capacity = int(inputs_cells["capacity"]) - transfer_capacity - fee
print("change capacity:", change_capacity, "shannon")
change_output = {"capacity":str(hex(change_capacity)),
               "lock":
                   {"code_hash": system_script["code_hash"],
                    "args": account_1["blake160"],
                    "hash_type":system_script["hash_type"]}
        }
print("change output:", change_output)
# set tx outputs and outputs_data
outputs = [transfer_output, change_output]
# keep same length with outputs
outputs_data = ["0x", "0x"]
print("outputs data:", outputs_data)
tx["outputs"] = outputs
tx["outputs_data"] = outputs_data

transfer output: {'capacity': '0x174876e800', 'lock': {'code_hash': '0x0274eb897aef04482f737d3fbee9c5983e510622c6cecd78f545e433ae6e70f0', 'args': '0xfd03fd824f9b777f3671ab776bd694cc76e3c09d', 'hash_type': 'data'}}
change capacity: 699995904000 shannon
change output: {'capacity': '0xa2fb01d800', 'lock': {'code_hash': '0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8', 'args': '0xfd03fd824f9b777f3671ab776bd694cc76e3c09d', 'hash_type': 'type'}}
outputs data: ['0x', '0x']


In [22]:
# build transaction and sign it
# clear example witnesses
tx["witnesses"] = ["0x"]
tx_s = json.dumps(tx)
print("transaction string:", tx_s)
# sign the tx
tx_witnesses = remote_server.sign(account_1["privkey"], tx_s)
print("witnesses:", tx_witnesses)

transaction string: {"version": "0x0", "cell_deps": [{"out_point": {"tx_hash": "0xace5ea83c478bb866edf122ff862085789158f5cbff155b7bb5f13058555b708", "index": "0x0"}, "dep_type": "dep_group"}], "header_deps": [], "inputs": [{"previous_output": {"tx_hash": "0x4f821ac07898673d2de8da1d9d6c797f43c82fa4d0537a05cec71e8d0a3d9fd8", "index": "0x1"}, "since": "0x0"}], "outputs": [{"capacity": "0x174876e800", "lock": {"code_hash": "0x0274eb897aef04482f737d3fbee9c5983e510622c6cecd78f545e433ae6e70f0", "args": "0xfd03fd824f9b777f3671ab776bd694cc76e3c09d", "hash_type": "data"}}, {"capacity": "0xa2fb01d800", "lock": {"code_hash": "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8", "args": "0xfd03fd824f9b777f3671ab776bd694cc76e3c09d", "hash_type": "type"}}], "outputs_data": ["0x", "0x"], "witnesses": ["0x"]}
witnesses: ['0x5500000010000000550000005500000041000000c6bdfc5facf74f4bf13d9c8dc1e6eb08f887302361adf03f5d5e7cf322ec76833fc4fcad7dc56ea8874bf49aa54d958d5129a0af7fc8bda33bb6ec2858186

In [23]:
# build raw transaction : transaction + witnesses
tx["witnesses"] = tx_witnesses
rtx_s = json.dumps(tx)
print("raw transaction string:", rtx_s)
transfer_tx_hash = remote_server.sendRawTransaction(rtx_s)
print("transafer transaction hash:", transfer_tx_hash)

raw transaction string: {"version": "0x0", "cell_deps": [{"out_point": {"tx_hash": "0xace5ea83c478bb866edf122ff862085789158f5cbff155b7bb5f13058555b708", "index": "0x0"}, "dep_type": "dep_group"}], "header_deps": [], "inputs": [{"previous_output": {"tx_hash": "0x4f821ac07898673d2de8da1d9d6c797f43c82fa4d0537a05cec71e8d0a3d9fd8", "index": "0x1"}, "since": "0x0"}], "outputs": [{"capacity": "0x174876e800", "lock": {"code_hash": "0x0274eb897aef04482f737d3fbee9c5983e510622c6cecd78f545e433ae6e70f0", "args": "0xfd03fd824f9b777f3671ab776bd694cc76e3c09d", "hash_type": "data"}}, {"capacity": "0xa2fb01d800", "lock": {"code_hash": "0x9bd7e06f3ecf4be0f2fcd2188b23f1b9fcc88e5d4b65a8637b17723bbda3cce8", "args": "0xfd03fd824f9b777f3671ab776bd694cc76e3c09d", "hash_type": "type"}}], "outputs_data": ["0x", "0x"], "witnesses": ["0x5500000010000000550000005500000041000000c6bdfc5facf74f4bf13d9c8dc1e6eb08f887302361adf03f5d5e7cf322ec76833fc4fcad7dc56ea8874bf49aa54d958d5129a0af7fc8bda33bb6ec285818646301"]}
transa

In [24]:
# wait for about 30s until see prev ouput about "transafer transaction hash"

# init for loop
loop_tx_hash = transfer_tx_hash
loop_capacity = transfer_capacity

In [28]:
# you can loop run this cell, but need wait for about 30s until see prev ouput about "loop tx hash"
# loop transfer to account 1 itself
print("loop capacity:", loop_capacity)
loop_tx = example_tx()
loop_tx_inputs = [{"previous_output": {"tx_hash": loop_tx_hash, "index": "0x0"}, "since": "0x0"}]
loop_capacity = loop_capacity - fee
loop_tx_outputs = [{"capacity":str(hex(loop_capacity)),
               "lock":
                   {"code_hash": always_sucess_info["code_hash"],
                    "args": account_1["blake160"],
                    "hash_type":always_sucess_info["hash_type"]}
        }]
loop_tx_outputs_data = ["0x"]
loop_tx_deps = [{"out_point": {"tx_hash": always_sucess_info["tx_hash"], "index": always_sucess_info["index"]}, "dep_type": always_sucess_info["dep_type"]}]
# always sucess lock scrip don't verify anything, so any witnesses is ok
loop_tx_witnesses = ["0x"]
# build raw transaction
loop_tx["cell_deps"] = loop_tx_deps
loop_tx["inputs"] = loop_tx_inputs
loop_tx["outputs"] = loop_tx_outputs
loop_tx["outputs_data"] = loop_tx_outputs_data
loop_tx["witnesses"] = loop_tx_witnesses
loop_rtx_s = json.dumps(loop_tx)
print("loop raw transaction string:", loop_rtx_s)
loop_tx_hash = remote_server.sendRawTransaction(loop_rtx_s)
print("loop tx hash:", loop_tx_hash)

loop capacity: 99995904000
loop raw transaction string: {"version": "0x0", "cell_deps": [{"out_point": {"tx_hash": "0xf232c8eca2e0db16898650ae924bd5a93ac274c8267e37295197174a5c09c858", "index": "0x0"}, "dep_type": "code"}], "header_deps": [], "inputs": [{"previous_output": {"tx_hash": "0xf1faad77bbea3b6fce842d7b9254a38088b3532c7382d8660117d03efb5efa68", "index": "0x0"}, "since": "0x0"}], "outputs": [{"capacity": "0x1748192800", "lock": {"code_hash": "0x0274eb897aef04482f737d3fbee9c5983e510622c6cecd78f545e433ae6e70f0", "args": "0xfd03fd824f9b777f3671ab776bd694cc76e3c09d", "hash_type": "data"}}], "outputs_data": ["0x"], "witnesses": ["0x"]}
loop tx hash: 0x9a629622f39089300d1d4048fab6a47ff3440ff2d0d0c760c79a0d92048f8ab4
